In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf

In [2]:
# Read the weather data from CSV
data = pd.read_csv("weatherHistory.csv")

In [3]:
features = data[['Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Wind Bearing (degrees)', 'Visibility (km)', 'Pressure (millibars)']]
target = data['Precip Type']

In [4]:
# Encode the target variable
label_encoder = LabelEncoder()
target = label_encoder.fit_transform(target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


In [8]:
#Instantiate the models
input_dim = X_train.shape[1]
output_dim = len(label_encoder.classes_)
generator = Generator(input_dim, output_dim)
discriminator = Discriminator(input_dim + output_dim, 1)  # Modify the input_dim and output_dim

# Define the loss functions and optimizers
criterion = nn.BCELoss()
gen_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
disc_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)

In [7]:
# Define the Generator and Discriminator models
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [9]:


# Training loop
num_epochs = 100
batch_size = 64

for epoch in range(num_epochs):
    for i in range(0, X_train.shape[0], batch_size):
        # Zero out the gradients
        gen_optimizer.zero_grad()
        disc_optimizer.zero_grad()

        # Get the batch data
        real_features = X_train[i:i + batch_size]
        real_targets = y_train[i:i + batch_size]

        # Generate fake data
        fake_targets = generator(real_features)
        fake_data = torch.cat((fake_targets, real_features), 1)  # Change the concatenation order

        # One-hot encode real targets
        real_targets_onehot = torch.zeros((real_targets.size(0), output_dim), dtype=torch.float32)
        real_targets_onehot.scatter_(1, real_targets.unsqueeze(1), 1)

        real_data = torch.cat((real_targets_onehot, real_features), 1)  # Change the concatenation order

        # Train the discriminator
        disc_real = discriminator(real_data)
        disc_fake = discriminator(fake_data)

        disc_loss_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_loss_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        disc_loss = (disc_loss_real + disc_loss_fake) / 2

        disc_loss.backward()
        disc_optimizer.step()

        # Train the generator
        fake_targets = generator(real_features)
        fake_data = torch.cat((fake_targets, real_features), 1)  # Change the concatenation order
        gen_output = discriminator(fake_data)
        gen_loss = criterion(gen_output, torch.ones_like(gen_output))

        gen_loss.backward()
        gen_optimizer.step()

    # Print the losses for monitoring
    print(f'Epoch {epoch+1}/{num_epochs}, Discriminator Loss: {disc_loss.item():.4f}, Generator Loss: {gen_loss.item():.4f}')


Epoch 1/100, Discriminator Loss: 0.6921, Generator Loss: 0.7968
Epoch 2/100, Discriminator Loss: 0.6955, Generator Loss: 0.6408
Epoch 3/100, Discriminator Loss: 0.6873, Generator Loss: 0.7076
Epoch 4/100, Discriminator Loss: 0.7019, Generator Loss: 0.6811
Epoch 5/100, Discriminator Loss: 0.6930, Generator Loss: 0.6296
Epoch 6/100, Discriminator Loss: 0.6973, Generator Loss: 0.6618
Epoch 7/100, Discriminator Loss: 0.7046, Generator Loss: 0.6118
Epoch 8/100, Discriminator Loss: 0.7057, Generator Loss: 0.7107
Epoch 9/100, Discriminator Loss: 0.6912, Generator Loss: 0.6953
Epoch 10/100, Discriminator Loss: 0.6874, Generator Loss: 0.7006
Epoch 11/100, Discriminator Loss: 0.6933, Generator Loss: 0.6593
Epoch 12/100, Discriminator Loss: 0.6949, Generator Loss: 0.7263
Epoch 13/100, Discriminator Loss: 0.6885, Generator Loss: 0.6956
Epoch 14/100, Discriminator Loss: 0.6980, Generator Loss: 0.6277
Epoch 15/100, Discriminator Loss: 0.6912, Generator Loss: 0.6626
Epoch 16/100, Discriminator Loss: 

In [28]:
generator.eval()
with torch.no_grad():
    test_targets = generator(X_test)
    test_targets = torch.argmax(test_targets, dim=1)
    accuracy = (test_targets == y_test).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.5177


In [38]:
generated_data = []
with torch.no_grad():
    for i in range(100):  # Generate 100 samples
        fake_targets = generator(X_test)
        fake_data = torch.cat((fake_targets, X_test), 1)
        generated_data.append(fake_data.numpy())

In [56]:
# Load the RNN model from the H5 file
rnn_model = load_model('weather_prediction_model.h5')  

2024-04-29 15:40:35.443013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 15:40:35.444043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 15:40:35.444742: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [60]:
# Select relevant features from the generated data array
relevant_generated_data_np = generated_data_np[:, :7]  # Select the first 7 features

# Reshape the generated data array to match the expected input shape (batch_size, 1, 7)
generated_data_reshaped = relevant_generated_data_np.reshape(-1, 1, 7)

# Make predictions using the loaded RNN model
with tf.device('/CPU:0'):
    rnn_output = rnn_model.predict(generated_data_reshaped)


   32/60285 [..............................] - ETA: 1:37   

2024-04-29 15:41:53.754979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 15:41:53.756289: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 15:41:53.757162: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

60285/60285 [==============================] - 95s 2ms/step
[0 0 0 ... 0 0 0]


In [64]:
print(rnn_output)

[[1.0000000e+00 3.4728345e-10]
 [1.0000000e+00 2.8320124e-33]
 [1.0000000e+00 7.2825305e-36]
 ...
 [1.0000000e+00 3.0120832e-20]
 [1.0000000e+00 3.9179632e-19]
 [1.0000000e+00 6.1567925e-12]]
